In [1]:
import pandas as pd
import numpy as np
import re
import csv
from string import ascii_lowercase
import random

### Concatenate all csv files into one DataFrame (and csv)

In [2]:
def create_df(file_path):
    '''
    Takes in file path and returns a dataframe
    Used for unknown delimiter
    '''
    # open csv file
    csv_ = open(file_path)

    # read in csv file
    read = csv.reader(csv_)

    # create empty list for dataframe
    file = []

    k = 0

    # read in each line
    for lines in read:
        # create/reset empty list for rows
        row = []

        # create/reset empty string for lyrics
        lyr = ''

        # create/reset accumulator
        j = 0

        for idx, i in enumerate(lines):

            # lyrics start at index = 4
            if idx > 3:
                # strip punctuations except ' (and maybe ,)
                i_no_punc = re.sub(r"[^\w\s']", '', i)

                # add string to lyr
                lyr += i_no_punc

                # add to accumulator
                j += 1

                # subtract 4 for the 5 columns we want to end up with - 1 
                # -1 since j accumulates before this statement
                if j == len(lines) - 4:
                    row.append(lyr)


            # append every line that isn't lyrics    
            else:
                row.append(i)

        
        # append rows to file for dataframe
        file.append(row)
        
    return pd.DataFrame(file[1:], columns = file[0])

In [3]:
# path to directory with all files
path = 'data/azlyrics-scraper/azlyrics_lyrics_'

# create list of all paths, optional: add path for file for bands who start with a number
all_paths = ['data/azlyrics-scraper/azlyrics_lyrics_19.csv']

# loop through every letter in the alphabet to 
for char in ascii_lowercase:
    # make new path to append
    pathway = path + char + '.csv'
    # append new path
    all_paths.append(pathway)

In [4]:
# create all_lyrics DF to concatenate other lyric dfs to
all_lyrics = create_df('data/azlyrics-scraper/azlyrics_lyrics_19.csv')

In [5]:
# for every path, create a dataframe and concatenate it to all_lyrics
# all_lyrics already has azlyrics_lyrics_19 in it, so 0th element is skipped
for _ in all_paths[1:]:
    df = create_df(_)
    all_lyrics = pd.concat([all_lyrics, df])

In [6]:
# reset index of all_lyrics
all_lyrics.reset_index(drop = True, inplace = True)

In [7]:
# save complete dataframe as csv
all_lyrics.to_csv(path + 'complete.csv')

### Creating punk rock/pop band subset of data

In [8]:
# list of pop punk bands - courtesy of Wikipedia (with some additions + deletions)
bands_punk = ["+44", "22 jacks", "5 seconds of summer", "999", "a",\
              "a day to remember", "the academy is...", "ace troubleshooter",\
              "acceptance", "afi", "against the current", "a change of pace",\
              "alien ant farm", "alkaline trio", "a loss for words", "all",\
              "the all-american rejects", "all time low", "allister",\
              "amber pacific", "american hi-fi", "army of freshmen", "anarbor", \
              "artist vs. poet", "as it is", "ash", "the ataris", "the audition",\
              "audio karate", "autopilot off", "avril lavigne", "beat crusaders",\
              "beat union", "better luck next time", "big drill car", "billy talent",\
              "blink-182", "bodyjar", "boris the sprinkler", "bomb the music industry!",\
              "the bouncing souls", "bowling for soup", "box car racer", "the boys",\
              "boys like girls", "bracket", "brand new", "broadside", "broadway calls",\
              "busted", "buzzcocks", "the cab", "carousel kings", "cartel", "cayetana",\
              "chasing morgan", "chixdiggit", "chunk! no, captain chunk!", "the click five",\
              "the copyrights", "count the stars", "courage my love", "crash romeo", "crimpshrine",\
              "cub", "cute is what we aim for", "daggermouth", "the dangerous summer",\
              "david crowder band", "the dead milkmen", "descendents", "the donnas", "the downtown fiction",\
              "driving east", "dum dums", "eat me raw", "every avenue", "eleventyseven", "elliot minor",\
              "energy", "the ergs!", "eternal boy", "eve 6", "even in blackouts", "everyday sunday",\
              "faber drive", "face to face", "falling in reverse", "fall out boy", "farewell", "fenix tx",\
              "fight fair", "finley", "fireworks", "flatfoot 56", "flop", "fluf", "fm static", "fonzie",\
              "forever came calling", "forever the sickest kids", "four year strong", "the friday night boys",\
              "generation x", "the get up kids", "ghoti hook", "go betty go", "go radio", "gob", "goldfinger",\
              "good charlotte", "goodnight nurse", "grayscale", "green day", "greyfield", "groovie ghoulies",\
              "guttermouth", "hagfish", "halifax", "handguns", "hangnail", "hawk nelson", "head injuries",\
              "hedley", "hey monday", "hey violet", "hidden in plain view", "the hi-fives", "the high court",\
              "the higher", "the hippos", "hit the lights", "home grown", "hot mulligan", "house of heroes",\
              "houston calls", "the huntingtons", "i call fives", "i fight dragons", "icon for hire",\
              "ivoryline", "the jam", "jawbreaker", "jeff rosenstock", "jimmy eat world", "june",\
              "just surrender", "karate high school", "kid down", "kids can't fly", "kids in glass houses",\
              "kids in the way", "killerpilze", "kiros", "kisschasy", "knuckle puck", "koopa", "lagwagon",\
              "latterman", "left front tire", "the leftovers", "the lemonheads", "less than jake", "lifesavors",\
              "lifetime", "light years", "like pacific", "the lillingtons", "limbeck", "lit", "lola ray",\
              "love you to death", "lucky 7", "lucky boys confusion", "ludo", "lustra", "mach pelican",\
              "magnapop", "the maine", "makeout", "man overboard", "manges", "marianas trench",\
              "masked intruder", "the matches", "mayday parade", "mcbusted", "mcfly",\
              "me first and the gimme gimmes", "melody fall", "mest", "the methadones", "midtown",\
              "millencolin", "mixtapes", "moose blood", "momoiro clover z", "motion city soundtrack",\
              "modern baseball", "the movielife", "the mr. t experience", "the muffs", "mxpx",\
              "my chemical romance", "neck deep", "nerf herder", "the new cities", "new found glory",\
              "new years day", "nofx", "noise by numbers", "not by choice", "no use for a name", "october fall",\
              "the offspring", "on my honor", "one ok rock", "orange", "over it", "paige", "parasites",\
              "paramore", "pardon us", "panic! at the disco", "patent pending", "permanent me",\
              "pee wee gaskins", "philmont", "pierce the veil", "plain white t's", "plinko", "pointed sticks",\
              "poor old lu", "the presidents of the united states of america", "propagandhi", "punchbuggy",\
              "punchline", "pup", "the queers", "quietdrive", "ramones", "rancid", "real friends",\
              "red city radio", "the red jumpsuit apparatus", "reggie and the full effect", "relient k",\
              "riddlin' kids", "roam", "rookie of the year", "rudi", "rufio", "run kid run", "saves the day",\
              "say anything", "saving aimee", "scenes from a movie", "scott murphy", "screeching weasel",\
              "seaway", "senses fail", "set it off", "set your goals", "shook ones", "short stack", "showoff",\
              "simple plan", "sleeping with sirens", "sleep on it", "slick shoes", "sludgeworth", "smash mouth",\
              "smoking popes", "something corporate", "son of dork", "the soviettes", "spanish love songs",\
              "sparks the rescue", "spazzys", "split habit", "sr-71", "stand atlantic", "the starting line",\
              "state champs", "stellar kart", "stereos", "steriogram", "stiff dylans", "story of the year",\
              "the story so far", "story untold", "student rick", "sugarcult", "sum 41", "supergrass",\
              "superman is dead", "the summer obsession", "the summer set", "sweet baby", "the swellers",\
              "tacocat", "taking back sunday", "teen idols", "teenage bottlerocket", "ten second epic",\
              "terrible things", "there for tomorrow", "these kids wear crowns", "this century",\
              "this time next year", "tigers jaw", "tilt", "title fight", "tonight alive", "transit",\
              "trash boat", "the travoltas", "treble charger", "trucks", "twenty twenty", "undercover",\
              "the undertones", "the unlovables", "unwritten law", "valencia", "the vandals", "vanilla sky",\
              "verona grove", "violent delight", "waterparks", "wavves", "wakefield", "we are the in crowd",\
              "the wedding", "weezer", "we the kings", "wheatus", "with confidence", "the wonder years",\
              "wstr", "yellowcard", "you, me, and everyone we know", "you me at six", "zebrahead",\
              "zolof the rock and roll destroyer", "menzingers, the", "anberlin", 'silverstein', 'rise against',\
              "the used"]

In [9]:
# restructure name of bands with "the" in front to match dataframe style
rest_bands = []
for i in bands_punk:
    # len of 'the ' is 4; check if first 4 characters are 'the '
    if i[:4] == 'the ':
        # take out 'the ' from band name and place at end of string
        restructured = i[4:] + ", the"
        # append to new list
        rest_bands.append(restructured)
    elif i[:2] == 'a ':
        # take out 'a ' from band name and place at end of string
        restructured = i[2:] + ", a"
        # append to new list
        rest_bands.append(restructured)
    else:
        # append name of bands without 'the ' in front
        rest_bands.append(i)

In [10]:
# create punk_df which has the punk rock/pop bands from our list of bands
punk_df = all_lyrics[all_lyrics.ARTIST_NAME.isin(rest_bands)]
punk_df.reset_index(drop = True, inplace = True)

In [11]:
# save dataframe to csv
punk_df.to_csv('data/punk_bands.csv')

In [12]:
punk_df

,ARTIST_NAME,ARTIST_URL,SONG_NAME,SONG_URL,LYRICS
0,5 seconds of summer,https://www.azlyrics.com/19/5secondsofsummer.html,gotta get out,https://www.azlyrics.com/lyrics/5secondsofsumm...,even when the sky is falling down even when th...
1,5 seconds of summer,https://www.azlyrics.com/19/5secondsofsummer.html,better man,https://www.azlyrics.com/lyrics/5secondsofsumm...,find me at a quarter to three cigarette in my ...
2,5 seconds of summer,https://www.azlyrics.com/19/5secondsofsummer.html,more,https://www.azlyrics.com/lyrics/5secondsofsumm...,if me and you are living in the same place why...
3,5 seconds of summer,https://www.azlyrics.com/19/5secondsofsummer.html,why won't you love me,https://www.azlyrics.com/lyrics/5secondsofsumm...,switching into airplane mode again we're not ...
4,5 seconds of summer,https://www.azlyrics.com/19/5secondsofsummer.html,woke up in japan,https://www.azlyrics.com/lyrics/5secondsofsumm...,i woke up in japan feeling low feeling lonely ...
...,...,...,...,...,...
2512,zebrahead,https://www.azlyrics.com/z/zebrahead.html,out of control,https://www.azlyrics.com/lyrics/zebrahead/outo...,i'm a mad man with a mission like a nightmare ...
2513,zebrahead,https://www.azlyrics.com/z/zebrahead.html,photographs,https://www.azlyrics.com/lyrics/zebrahead/phot...,i am alive i am awake not like i'm not aware i...
2514,zebrahead,https://www.azlyrics.com/z/zebrahead.html,politics,https://www.azlyrics.com/lyrics/zebrahead/poli...,firewall school halls cop cars protecting you ...
2515,zebrahead,https://www.azlyrics.com/z/zebrahead.html,with legs like that,https://www.azlyrics.com/lyrics/zebrahead/with...,here she comes again like good medicine every ...


In [13]:
# create seed for reusability
random.seed(a = 11)

#create list of subset of punk bands
subset_punk = []

# subset only using 1/3 bands found in the dataframe
while len(subset_punk) < round(len(np.unique(punk_df.ARTIST_NAME))/3):
    # randomly choose a band from the list
    chosen = random.choice(np.unique(punk_df.ARTIST_NAME))
    # if band not in the list, append the band to the list
    if chosen not in subset_punk:
        subset_punk.append(chosen)

In [14]:
subset_punk_df = all_lyrics[all_lyrics.ARTIST_NAME.isin(subset_punk)]
subset_punk_df.reset_index(drop = True, inplace = True)

In [15]:
# save dataframe to csv
subset_punk_df.to_csv('data/subset_punk_bands.csv')

#Pseudo Code for splitting lyrics (based on commas)
```python
split_lyrics_lst= []
for row in punk df:
    split_lyr = row.split(', ')
    rows = []
    for line in split_lyr:
        rows.append(name, song name, line)
        
    split_lyrics_lst.append(rows)
```

## Creating a model

### Setting up the data

In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Input, Embedding, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

In [17]:
# "randomly" choose one band for model
yellowcard = punk_df.LYRICS[punk_df.ARTIST_NAME == 'yellowcard']
tokenizer = Tokenizer()

tokenizer.fit_on_texts(yellowcard)

token_seq = tokenizer.texts_to_sequences(yellowcard)

n_gram_seq = []
# for every line in tokenized sequences
for line in token_seq:
    # used to append the token_seq starting from 0th element to 1st element
    for length in range(2, len(line)):
        n_gram_seq.append(line[:length])

In [18]:
# create padded sequences
n_gram_seq_padded = pad_sequences(n_gram_seq, maxlen = 11)

In [19]:
n_gram_seq_padded[:10]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0, 704,   3],
       [  0,   0,   0,   0,   0,   0,   0,   0, 704,   3, 108],
       [  0,   0,   0,   0,   0,   0,   0, 704,   3, 108, 158],
       [  0,   0,   0,   0,   0,   0, 704,   3, 108, 158,  14],
       [  0,   0,   0,   0,   0, 704,   3, 108, 158,  14,   9],
       [  0,   0,   0,   0, 704,   3, 108, 158,  14,   9, 204],
       [  0,   0,   0, 704,   3, 108, 158,  14,   9, 204,   1],
       [  0,   0, 704,   3, 108, 158,  14,   9, 204,   1,  63],
       [  0, 704,   3, 108, 158,  14,   9, 204,   1,  63, 113],
       [704,   3, 108, 158,  14,   9, 204,   1,  63, 113,   8]],
      dtype=int32)

In [20]:
# create labels by using One Hot Encoding 
labels = to_categorical(n_gram_seq_padded[:,-1:])
X = n_gram_seq_padded[:,:-1]

train_size = round(n_gram_seq_padded.shape[0]*0.8)

# create test and train
y_train = labels[:train_size:]
y_test = labels[train_size:,:]

x_train = X[:train_size,:]
x_test = X[train_size:,:]

# find largest vocab size in padded sequence; this is input size
vocab_size = max([w for sentence in n_gram_seq_padded for w in sentence]) + 1

### Creating the model

In [21]:
model = keras.Sequential()
model.add(Embedding(input_dim = vocab_size, output_dim = 64, input_length = 11))

# add bidirectional lstm layer
model.add(Bidirectional(LSTM(128, return_sequences = True)))

# add dropout layer
model.add(Dropout(0.2))

# add another LSTM layer
model.add(LSTM(64))

model.add(Dense(round(vocab_size/2), activation = 'relu'))

# potentially add embedding layer

model.add(Dense(vocab_size, activation = 'softmax'))

#compile model
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = 'accuracy')

# Stops training if validation loss doesn't improve
callback = EarlyStopping(monitor = 'val_loss', patience = 3)

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 11, 64)            74304     
_________________________________________________________________
bidirectional (Bidirectional (None, 11, 256)           197632    
_________________________________________________________________
dropout (Dropout)            (None, 11, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                82176     
_________________________________________________________________
dense (Dense)                (None, 580)               37700     
_________________________________________________________________
dense_1 (Dense)              (None, 1161)              674541    
Total params: 1,066,353
Trainable params: 1,066,353
Non-trainable params: 0
______________________________________________

In [23]:
tf.config.run_functions_eagerly(True)

In [24]:
model.fit(x_train, y_train, epochs = 20, validation_data = (x_test, y_test))

Epoch 1/20


/Users/belindanguyen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


281/281 [==============================] - 40s 143ms/step - loss: 6.1875 - accuracy: 0.0468 - val_loss: 5.8822 - val_accuracy: 0.0498
Epoch 2/20
281/281 [==============================] - 40s 143ms/step - loss: 5.4612 - accuracy: 0.0540 - val_loss: 5.8352 - val_accuracy: 0.0587
Epoch 3/20
281/281 [==============================] - 39s 138ms/step - loss: 5.2003 - accuracy: 0.0564 - val_loss: 5.8996 - val_accuracy: 0.0405
Epoch 4/20
281/281 [==============================] - 42s 149ms/step - loss: 4.9428 - accuracy: 0.0774 - val_loss: 6.0174 - val_accuracy: 0.0628
Epoch 5/20
281/281 [==============================] - 38s 134ms/step - loss: 4.6774 - accuracy: 0.0946 - val_loss: 6.1232 - val_accuracy: 0.0619
Epoch 6/20
281/281 [==============================] - 38s 137ms/step - loss: 4.4150 - accuracy: 0.1183 - val_loss: 6.3707 - val_accuracy: 0.0694
Epoch 7/20
281/281 [==============================] - 41s 146ms/step - loss: 4.0891 - accuracy: 0.1653 - val_loss: 6.5457 - val_accuracy: 0.0

In [25]:
def generate_lyrics(prompt, length):
    '''
    prompt: string of lyrics
    length: length of lyrics that is wanted (includes prompt)
    '''
    # edge case; if prompt is as long as the length wanted
    if len(prompt.split(' ')) == length:
        return prompt
    else:
        for _ in range(length - len(prompt.split(' '))):
            
            token_list = tokenizer.texts_to_sequences([prompt])[0]
            token_padded = pad_sequences([token_list], maxlen = 11)
            
            # get predicted probability for each word
            predicted_probs = model.predict(token_padded)[0]
            
            # find max probability of the each word
            word_choice = predicted_probs.argmax()
            
            # find out what the word is
            output_word = tokenizer.index_word[word_choice]
            
            # add word to the prompt
            prompt += ' ' + output_word
        return prompt

In [26]:
generate_lyrics("eyes are feeling heavy but they never seem to close", 20)

"eyes are feeling heavy but they never seem to close the same then i've always been here on the while"

In [27]:
generate_lyrics("there's a place off ocean avenue", 20)

"there's a place off ocean avenue you even here you've always climb up and i learned to want to say"

In [28]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk

In [29]:
# associate tags with a number
tags_index = dict()
for idx, val in enumerate(np.unique(punk_df.ARTIST_NAME)):
    tags_index[val] = idx

In [53]:
tagged_docs = []

for idx, val in tags_index.items():
    # create list for all lyrics
    all_lyrics = []
    # for all the lyrics of the songs the artist made
    for i in punk_df.LYRICS[punk_df.ARTIST_NAME == idx].index:
        # tokenize and append each song's lyrics to list
        all_lyrics.append(nltk.word_tokenize(punk_df.LYRICS[punk_df.ARTIST_NAME == idx][i]))
    # tag all tokenized lyrics with artist's name and append to tagged_docs
    tagged_docs.append(TaggedDocument(all_lyrics, tags = idx))

In [55]:
model_dbow = Doc2Vec(epochs = 20, hs = 1, alpha = 0.001)

In [34]:
model_dbow.build_vocab(documents = [x for x in tagged_docs])

TypeError: 'int' object is not iterable

In [ ]:
type([docs for docs in tagged_docs])

In [35]:
data = ["I love machine learning. Its awesome.",
        "I love coding in python",
        "I love building chatbots",
        "they chat amagingly well"]

tagged_data = [TaggedDocument(words=nltk.word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

In [40]:
max_epochs = 100
vec_size = 20
alpha = 0.025

m = Doc2Vec()
  
m.build_vocab(tagged_docs[0])

AttributeError: 'list' object has no attribute 'words'

In [37]:
type(tagged_data)

list

In [38]:
tagged_data

[TaggedDocument(words=['i', 'love', 'machine', 'learning', '.', 'its', 'awesome', '.'], tags=['0']),
 TaggedDocument(words=['i', 'love', 'coding', 'in', 'python'], tags=['1']),
 TaggedDocument(words=['i', 'love', 'building', 'chatbots'], tags=['2']),
 TaggedDocument(words=['they', 'chat', 'amagingly', 'well'], tags=['3'])]

In [ ]:
gensim.__version__